In [2]:
import pandas as pd
import matplotlib.pyplot as plt

# Caminho para o arquivo
arquivo = 'Afetacao_municipios.xlsx'

# Carrega as planilhas
xls = pd.ExcelFile(arquivo)
df_claro = xls.parse('Claro')
df_tim = xls.parse('Tim')
df_vivo = xls.parse('Vivo')

# Função para preparar os dados de cada operadora
def preparar_dados(df, operadora, coluna_afetacao):
    df.columns = df.columns.str.strip()  # remove espaços extras
    df = df.rename(columns={df.columns[0]: 'Data', coluna_afetacao: 'Afetacao', 'Nº_Municipios': 'Quantidade'})
    df['Data'] = pd.to_datetime(df['Data'], dayfirst=True)
    df = df[df['Afetacao'] != 'Não atende o município']
    df['Operadora'] = operadora
    return df[['Data', 'Afetacao', 'Quantidade', 'Operadora']]

# Preparar dados das três operadoras
claro = preparar_dados(df_claro, 'Claro', 'Afetacao_Claro')
tim = preparar_dados(df_tim, 'Tim', 'Afetacao_Tim')
vivo = preparar_dados(df_vivo, 'Vivo', 'Afetacao_Vivo')

# Junta os dados
dados = pd.concat([claro, tim, vivo], ignore_index=True)

# Agrupa por data, operadora e tipo de afetação
grupo = dados.groupby(['Data', 'Operadora', 'Afetacao'])['Quantidade'].sum().reset_index()

# Lista de tipos de afetação em ordem desejada
tipos_afetacao = ['Sem afetação', 'Afetação reduzida', 'Afetação elevada', 'Afetação total']
grupo['Afetacao'] = pd.Categorical(grupo['Afetacao'], categories=tipos_afetacao, ordered=True)

# Geração de gráficos por tipo de afetação
fig, axs = plt.subplots(nrows=4, figsize=(12, 16), sharex=True)
cores = {'Claro': 'blue', 'Tim': 'green', 'Vivo': 'red'}

for i, tipo in enumerate(tipos_afetacao):
    ax = axs[i]
    for operadora in ['Claro', 'Tim', 'Vivo']:
        dados_plot = grupo[(grupo['Operadora'] == operadora) & (grupo['Afetacao'] == tipo)]
        ax.plot(dados_plot['Data'], dados_plot['Quantidade'], label=operadora, color=cores[operadora], marker='o')
    ax.set_title(f"Tipo de Afetação: {tipo}")
    ax.set_ylabel("Nº de Municípios")
    ax.legend()

plt.xlabel("Data")
plt.suptitle("Afetação por Dia e por Operadora", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.grid(True)
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'Afetacao_municipios.xlsx'